In [1]:
import numpy as np
import pickle
import math
import os

%tensorflow_version 1.x
import tensorflow as tf
import keras

from keras.models import Model
from keras.layers import Dense, Dropout, Conv2D, Reshape, MaxPooling2D
from keras.layers import Concatenate, Lambda, Dot, BatchNormalization, Flatten

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.14`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


Using TensorFlow backend.


In [0]:
# functions to execute dominant sets clustering algorithm
# http://homepage.tudelft.nl/3e2t5/HungKrose_ICMI2011.pdf

# fills an n_people x n_people matrix with affinity values.
def learned_affinity(preds, n_people):
	A = np.zeros((n_people, n_people))
	idx = 0

	for i in range(n_people):
		for j in range(n_people):
			if i == j: continue
			A[i,j] += preds[idx]/2
			A[j,i] += preds[idx]/2
			idx += 1

	return A

# d-sets function k
def k(S, i, A):
	sum_affs = 0
	for j in range(len(S)):
		if S[j]: sum_affs += A[i,j]

	return 1/np.sum(S) * sum_affs

# d-sets function phi
def phi(S,i,j,A):
	return A[i,j] - k(S,i,A)

# d-sets function weight
def weight(S, i, A):
	if np.sum(S) == 1:
		return 1
	else:
		R = S.copy()
		R[i] = False
		sum_weights = 0
		for j in range(len(R)):
			if R[j]:
				sum_weights += phi(R,j,i,A) * weight(R,j,A)
				return sum_weights

## optimization function
def f(x, A):
	return np.dot(x.T, np.dot(A, x))

## iteratively finds vector x which maximizes f
def vector_climb(A, allowed, n_people, thres=1e-5):
	x = np.random.uniform(0,1,n_people)
	x = np.multiply(x, allowed)
	eps = 10
	n = 10

	while (eps > 1e-15):
		p = f(x,A)
		x = np.multiply(x, np.dot(A,x)) / np.dot(x, np.dot(A,x))
		n = f(x,A)
		eps = abs(n-p)

	groups = x > thres

	for i in range(n_people):
		if not allowed[i]:
			if weight(groups,i,A) > 0.0:
				return []

	return groups

def process_groups(bool_groups, n_people):
    groups = []
    for bool_group in bool_groups:
        group = []
        for i in range(n_people):
            if (bool_group[i]): group.append("ID_00" + str(i+1))
        if(len(group)>1): groups.append(group)
    return groups

# Finds vectors x of people which maximize f. Then removes those people and repeats
# main method
def ds(preds, n_people):
    allowed = np.ones(n_people)
    groups = []

    A = learned_affinity(preds, n_people)

    while (np.sum(allowed) > 1):
        A[allowed == False] = 0
        A[:,allowed == False] = 0

        if (np.sum(np.dot(allowed,A)) == 0): break
        x = vector_climb(A, allowed, n_people, thres=1e-5)
        if len(x) == 0: break
        groups.append(x)

        allowed = np.multiply(x == False, allowed)

    return process_groups(groups, n_people)

In [0]:
def calc_f1(X, Y, times, preds, thres):
    results = np.array([0.0, 0.0])
    for i in range(len(times)-1):
        start, stop = int(times[i]), int(times[i+1])
        num_people = int(np.sqrt(stop-start))+1

        pred = ds(preds[start:stop], num_people)
        truth = ds(Y[start:stop], num_people)

        TF, FN, FP, P, R = indiv_f1(pred, truth, thres)
        results += np.array([P, R])

    results /= (len(times) - 1)
    P, R = results

    f1 = 2 * P * R / (P + R)
    return P, R, f1

## calculates true positives, false negatives, and false positives
## given the guesses, the true groups, and the threshold T
def indiv_f1(pred, truth, T):
    TP, FN, FP = 0, 0, 0
    n_true_groups = len(truth)
    n_pred_groups = len(pred)

    for true_group in truth:
        for pred_group in pred:
            n_found = 0
            for person in pred_group:
                if person in true_group:
                    n_found += 1

            n_total = max(len(true_group), len(pred_group))
            acc = float(n_found) / n_total
            if acc>=T: TP += 1

    if n_true_groups == 0 and n_pred_groups == 0:
        return [0, 0, 0, 1, 1]
    elif n_true_groups == 0:
        return [0, n_pred_groups, 0, 0, 1]
    elif n_pred_groups == 0:
        return [0, 0, n_true_groups, 1, 0]
    else:
        FP = n_pred_groups - TP
        FN = n_true_groups - TP
        P = float(TP) / (TP + FP) #precision
        R = float(TP) / (TP + FN) #recall
        return TP, FN, FP, P, R

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

'''
standard feature set
downloaded = drive.CreateFile({'id':"1d7EKeFXGlcKA_9XplIkGrIRQS-eGM8_w"})   # replace the id with id of file you want to access
downloaded.GetContentFile('train.p')        # replace the file name with your file

downloaded = drive.CreateFile({'id':"18rLSsWTWybeSs0r0S73CoiitgcLH3xZJ"})   # replace the id with id of file you want to access
downloaded.GetContentFile('test.p')        # replace the file name with your file

downloaded = drive.CreateFile({'id':"1pmkvRbnl7zvgb6scazItMITlp2GyMQlL"})   # replace the id with id of file you want to access
downloaded.GetContentFile('val.p')        # replace the file name with your file
'''

downloaded = drive.CreateFile({'id':"1xhR_ptuiC1mprelvE2Sfb4BDcwMAlTy0"})   # replace the id with id of file you want to access
downloaded.GetContentFile('train.p')        # replace the file name with your file

downloaded = drive.CreateFile({'id':"1wsFIqrosQx4CHFbH476rQz-WBLJqRG_K"})   # replace the id with id of file you want to access
downloaded.GetContentFile('test.p')        # replace the file name with your file

downloaded = drive.CreateFile({'id':"1RtYY820FZTmHvkC6VSNIiZRWi7XW8Yc1"})   # replace the id with id of file you want to access
downloaded.GetContentFile('val.p')        # replace the file name with your file

def load_matrix(file):
    with open(file, 'rb') as f:
        return pickle.load(f)

def load_data(dataset):
    path = "./datasets/" + str(dataset) + "/processed"
    train = load_matrix('train.p')
    test = load_matrix('test.p')
    val = load_matrix('val.p')
    return train, test, val

In [0]:
class ValLoss(keras.callbacks.Callback):

    def __init__(self, val):
        super(ValLoss, self).__init__()
        self.X, self.Y, self.times = val

        self.best_model = None
        self.best_val_mse = float("inf")
        self.best_epoch = -1

        self.val_f1 = {"f1s": [], "best_f1": float('-inf')}
        self.val_f2_3 = {"f1s": [], "best_f1": float('-inf')}

        self.val_losses = []
        self.train_losses = []

        self.val_mses = []
        self.train_mses = []

    def on_epoch_end(self, epoch, logs={}):
        print(logs)
        if logs['val_mean_squared_error'] < self.best_val_mse:
            self.best_model = self.model
            self.best_val_mse = logs['val_mean_squared_error']
            self.best_epoch = epoch

        f_1 = calc_f1(self.X, self.Y, self.times, self.model.predict(self.X), 1)[2]
        f_2_3 = calc_f1(self.X, self.Y, self.times, self.model.predict(self.X), 2/3)[2]

        for f_1, obj in [(f_1, self.val_f1), (f_2_3, self.val_f2_3)]:
            if f_1 > obj['best_f1']:
                obj['best_f1'] = f_1
                obj['epoch'] = epoch
                obj['model'] = self.model
            obj['f1s'].append(f_1)

        self.val_losses.append(logs['val_loss'])
        self.train_losses.append(logs['loss'])
        self.val_mses.append(logs['val_mean_squared_error'])
        self.train_mses.append(logs['mean_squared_error'])

def write_history(file_name, history, test):
    file = open(file_name, 'w+')

    file.write("best_val: " + str(history.best_val_mse))
    file.write("\nepoch: " + str(history.best_epoch))

    file.write("\nbest_val_f1_1: " + str(history.val_f1['best_f1']))
    file.write("\nepoch: " + str(history.val_f1['epoch']))

    X_test, Y_test, times_test = test
    preds = history.val_f1['model'].predict(X_test)
    p23, r23, f23 = calc_f1(X_test, Y_test, times_test, preds, 2/3)
    p1, r1, f1 = calc_f1(X_test, Y_test, times_test, preds, 1)

    file.write("\ntest_f1s: " + str(f23) + " " + str(f1))
    file.write('\nprecisions: ' + str(p23) + " " + str(p1))
    file.write('\nrecalls: ' + str(r23) + " " + str(r1))

    file.write("\nbest_val_f1_2/3: " + str(history.val_f2_3['best_f1']))
    file.write("\nepoch: " + str(history.val_f2_3['epoch']))

    X_test, Y_test, times_test = test
    preds = history.val_f2_3['model'].predict(X_test)
    p23, r23, f23 = calc_f1(X_test, Y_test, times_test, preds, 2/3)
    p1, r1, f1 = calc_f1(X_test, Y_test, times_test, preds, 1)

    file.write("\ntest_f1s: " + str(f23) + " " + str(f1))
    file.write('\nprecisions: ' + str(p23) + " " + str(p1))
    file.write('\nrecalls: ' + str(r23) + " " + str(r1))

    file.write("\ntrain loss:")
    for loss in history.train_losses:
        file.write('\n' + str(loss))

    file.write("\nval loss:")
    for loss in history.val_losses:
        file.write('\n' + str(loss))

    file.write("\ntrain mse:")
    for loss in history.train_mses:
        file.write('\n' + str(loss))

    file.write("\nval mse:")
    for loss in history.val_mses:
        file.write('\n' + str(loss))

    file.write("\nval 1 f1:")
    for f1 in history.val_f1['f1s']:
        file.write('\n' + str(f1))

    file.write("\nval 2/3 f1:")
    for f1 in history.val_f2_3['f1s']:
        file.write('\n' + str(f1))

    file.close()

def conv(filters, reg, name=None):
    return Conv2D(filters=filters, kernel_size=1, padding='valid', kernel_initializer="he_normal",
        use_bias='True', kernel_regularizer=reg, activation=tf.nn.relu, name=name)

def build_model(reg_amt, drop_amt, max_people, num_features, global_filters, individual_filters, combined_filters):

    group_inputs = keras.layers.Input(shape=(1, max_people, num_features))
    pair_inputs = keras.layers.Input(shape=(1, 2, num_features))

    reg = keras.regularizers.l2(reg_amt)

    y = pair_inputs

    # Dyad Transform
    for filters in individual_filters:
        y = conv(filters, reg)(y)
        y = Dropout(drop_amt)(y)
        y = BatchNormalization()(y)

    y_0 = Lambda(lambda input: tf.slice(input, [0, 0, 0, 0], [-1, -1, 1, -1]))(y)
    y_1 = Lambda(lambda input: tf.slice(input, [0, 0, 1, 0], [-1, -1, 1, -1]))(y)

    x = group_inputs

    # Context Transform
    for filters in global_filters:
        x = conv(filters, reg)(x)
        x = Dropout(drop_amt)(x)
        x = BatchNormalization()(x)

    x = MaxPooling2D(name="global_pool", pool_size=[1, max_people], strides=1, padding='valid')(x)
    x = Dropout(drop_amt)(x)
    x = BatchNormalization()(x)
    x_flat = Flatten()(x)

    concat = Concatenate(name='concat')([x_flat, Flatten()(y_0), Flatten()(y_1)])

    # Final MLP from paper
    for filters in combined_filters:
        concat = Dense(units=filters, use_bias='True', kernel_regularizer=reg, activation=tf.nn.relu,
            kernel_initializer="he_normal")(concat)
        concat = Dropout(drop_amt)(concat)
        concat = BatchNormalization()(concat)

    # final pred
    affinity = Dense(units=1, use_bias="True", kernel_regularizer=reg, activation=tf.nn.sigmoid,
        name='affinity', kernel_initializer="glorot_normal")(concat)

    model = Model(inputs=[group_inputs, pair_inputs], outputs=affinity)

    opt = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, decay=1e-5, amsgrad=False, clipvalue=0.5)
    model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['mean_squared_error'])

    return model


In [0]:
# constructs a model, trains it with early stopping based on validation loss, and then
# saves the output to a .txt file.
def train_and_save_model(global_filters, individual_filters, combined_filters,
    train, val, test, model_path, epochs=600, reg=0.0000001, dropout=.35):

    # ensures repeatability
    tf.set_random_seed(0)
    np.random.seed(0)

    num_train, _, max_people, num_features = train[0][0].shape

    # save achitecture
    if not os.path.isdir(model_path): os.makedirs(model_path)
    file = open(model_path + '/architecture.txt', 'w+')
    file.write("global: " + str(global_filters) + "\nindividual: " +
        str(individual_filters) + "\ncombined: " + str(combined_filters) +
        "\nreg= " + str(reg) + "\ndropout= " + str(dropout))

    best_val_mses = []
    best_val_f1 = []
    best_val_f2_3 = []

    X_train, Y_train, times_train = train
    X_test, Y_test, times_test = test
    X_val, Y_val, times_val = val

    # build model
    model = build_model(reg, dropout, max_people, num_features,
        global_filters, individual_filters, combined_filters)

    # train model val_mse
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
    history = ValLoss(test) #custom callback implemented above

    #os.system('cls') #hides annoying warnings
    model.fit(X_train, Y_train, epochs=epochs, batch_size=1024,
        validation_data=(X_val, Y_val), callbacks=[history, early_stop])

    best_val_mses.append(history.best_val_mse)
    best_val_f1.append(history.val_f1['best_f1'])
    best_val_f2_3.append(history.val_f2_3['best_f1'])

    # save model
    write_history(model_path + '/results.txt', history, test)
    history.val_f1['model'].save(model_path + '/model.h5')

    file.write("\n\nbest overall val loss: " + str(min(best_val_mses)))
    file.write("\n\nbest overall f1 1: " + str(max(best_val_f1)))
    file.write("\n\nbest overall f1 2/3: " + str(max(best_val_f2_3)))
    file.close()

In [7]:
global_filters = [16, 128, 512]
individual_filters = [32]
combined_filters = [1024, 256, 256]
epochs = 600
reg = 7.943282347242822e-05
dropout = 0.15

train, test, val = load_data("cocktail")
model_path = "./models/cocktail_expanded/model2"

train_and_save_model(global_filters, individual_filters, combined_filters,
    train, val, test, model_path, epochs, reg, dropout)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.










Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 13440 samples, validate on 1920 samples
Epoch 1/600
13440/13440 [==============================] - 4s 309us/step - loss: 1.1897 - mean_squared_error: 0.2916 - val_loss: 1.1476 - val_mean_squared_error: 0.2780
{'val_loss': 1.1476022283236185, 'val_mean_squared_error': 0.2779996673266093, 'loss': 1.1897113345918202, 'mean_squared_error': 0.29157915541103907}
Epoch 2/600
13440/13440 [==============================] - 0s 31us/step - loss: 1.0993 - mean_squared_error: 0.2572 - val_loss: 1.0510 - val_mean_squared_error: 0.2448
{'val_loss': 1.050971007347107, 'val_mean_squared_error': 0.2448214148481687, 'loss': 1.099290867078872, 'mean_squared_error': 0.25716616426195416}
Epoch 3/600
13440/13440 [==============================] - 0s 31us/step - loss: 1.0387 - me